tgb - 12/20/2021- The goal is to directly derive the climate-invariant dataset using the custom generator to avoid inconsistencies in the formulation of the relative humidity, plume buoyancy, and scaled latent heat flux rescalings. This dataset can then be used for the causal discovery project led by Nando Iglesias. 

# Imports

In [1]:
from cbrain.climate_invariant import *
from cbrain.climate_invariant_utils import *

/nfspool-0/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog


In [2]:
path_data = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/'

# Define data generators

## Below is how we would build a standard or "brute-force" data generator `train_gen_BF`.

1. We would first specify the input variables `in_vars`, the output variables `out_vars`, and the path of the training set `path_train`. 

In [3]:
in_vars = ['QBP','TBP','PS','SOLIN','SHFLX','LHFLX'] # We take the large-scale climate state as inputs
out_vars = ['PHQ','TPHYSTND','FSNT','FSNS','FLNT','FLNS', 'PRECT'] # and we output the response of clouds/storms to these climate conditions
path_train = path_data + 'Aqua_0K_withVBP/2021_09_02_TRAIN_For_Nando.nc'

2. To make sure all outputs have the same units (in our case W/m$^2 $), we multiply the raw outputs by the right physical constants, stored in a dictionary called `scale_dict`. 

In [4]:
import pickle
scale_dict = pickle.load(open(path_data+'CIML_Zenodo/009_Wm2_scaling.pkl','rb'))

3. We scale the inputs to [-1,1] by subtracting their mean before dividing them by their range. The means and ranges are stored in a normalization file stored in `path_input_norm`. 

In [5]:
path_input_norm = path_data + '001_norm.nc'

4. We are now ready to build our first data generator!

In [6]:
N_batch = 8192

In [7]:
train_gen_BF = DataGeneratorCI(
    data_fn = path_train,
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = path_input_norm,
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    shuffle = False,
    batch_size=N_batch
)

## Now, we would like to build a "climate-invariant" data generator `train_gen_CI`, which requires a few more steps

### First, we have to create one standard generator per input rescaling. This will help us renormalize the inputs to [-1,1] every time we feed them to the neural network. 

1. First, let's define the path to the three normalization files for the three input rescalings:
Relative humidity `RH`, plume buoyancy `BMSE`, and normalized latent heat flux `LHF_nsDELQ` 

In [8]:
path_norm_RH = path_data + '2021_02_01_NORM_O3_RH_small.nc'
path_norm_BMSE = path_data + '2021_06_16_NORM_BMSE_small.nc'
path_norm_LHF_nsDELQ = path_data + '2021_02_01_NORM_O3_LHF_nsDELQ_small.nc'

2. We can now define one data generator per input rescaling

In [9]:
def train_gen_rescaling(input_rescaling):
    return DataGeneratorCI(
        data_fn = path_train,
        input_vars = input_rescaling,
        output_vars = out_vars,
        norm_fn = path_input_norm,
        input_transform = ('mean', 'maxrs'),
        output_transform = scale_dict)

In [10]:
train_gen_RH = train_gen_rescaling(in_vars)
train_gen_BMSE = train_gen_rescaling(in_vars)
train_gen_LHF_nsDELQ = train_gen_rescaling(in_vars)

### Then, the normalization factors of these generators can be combined to form a "climate-invariant" data generator `train_gen_CI`

In [11]:
train_gen_CI = DataGeneratorCI(
    data_fn = path_train,
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = path_input_norm,
    input_transform = ('mean','maxrs'),
    output_transform = scale_dict,
    shuffle = False,
    batch_size=N_batch,
    Qscaling = 'RH',
    Tscaling = 'BMSE',
    LHFscaling = 'LHF_nsDELQ',
    hyam=hyam, hybm=hybm, # Arrays to define mid-levels of hybrid vertical coordinate
    inp_sub_Qscaling=train_gen_RH.input_transform.sub, # What to subtract from RH inputs
    inp_div_Qscaling=train_gen_RH.input_transform.div, # What to divide RH inputs by
    inp_sub_Tscaling=train_gen_BMSE.input_transform.sub,
    inp_div_Tscaling=train_gen_BMSE.input_transform.div,
    inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
    inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div
)

# Regenerate the scaled dataset

## Create new training file

In [ ]:
path_train = path_data + 'Aqua_0K_withVBP/2021_09_02_TRAIN_For_Nando.nc'

In [ ]:
train_raw = xr.open_dataset(path_train)

### var_names

In [ ]:
train_raw['var_names'].values

In [ ]:
train_raw_CI = train_raw.copy()

In [ ]:
var_names_CI = train_raw_CI['var_names'].values
for i in range(60):
    if i<30: var_names_CI[i] = 'RH'
    else: var_names_CI[i] = 'BMSE'

In [ ]:
var_names_CI[93] = 'LHF_nsDELQ'

In [ ]:
train_raw_CI.assign_coords({'var_names':var_names_CI})

### vars

In [ ]:
train_gen_CI.output_transform.scale.shape

In [ ]:
train_gen_CI[0][0].shape

In [ ]:
train_raw_CI['vars'][0].values.shape

In [ ]:
train_gen_BF_0_pu = (train_gen_BF[0][0]*train_gen_BF.input_transform.div+train_gen_BF.input_transform.sub)

In [ ]:
train_gen_BF_0_pu[1000][:30]

In [ ]:
train_raw_CI['vars'].shape

In [ ]:
train_raw_CI['vars'][1000][:30].values

In [ ]:
train_gen_CI_0_pu = (train_gen_CI[0][0]*train_gen_CI.input_transform.div+train_gen_CI.input_transform.sub)

In [ ]:
train_gen_CI_0_pu[1000][:30]

In [ ]:
new_values = np.zeros(train_raw_CI['vars'].shape)

In [ ]:
for ibatch in range((train_gen_CI.n_samples)//N_batch):
    if ibatch % 10==0: print('progress=','%2.2f' % (100*ibatch/((train_gen_CI.n_samples)//N_batch)),
                              '%','               ',end='\r')
    train_gen_CI_pu = (train_gen_CI[ibatch][0]*train_gen_CI.input_transform.div+\
                       train_gen_CI.input_transform.sub)
    new_values[ibatch*N_batch:((1+ibatch)*N_batch),:] = np.concatenate(
        (train_gen_CI_pu[:,:60],
        train_raw_CI['vars'][ibatch*N_batch:((1+ibatch)*N_batch),60:90],
        train_gen_CI_pu[:,60:],
        train_raw_CI['vars'][ibatch*N_batch:((1+ibatch)*N_batch),94:]),
        axis=1
    )

In [ ]:
new_values.shape

In [ ]:
train_raw_CI['vars'].values = new_values

### Save new training dataset

In [ ]:
path_save_dir = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/Aqua_0K_ClimInv_withVBP/'

In [ ]:
train_raw_CI.to_netcdf(path_save_dir+'2021_12_22_TRAIN_For_Nando_CI.nc',mode='w')

In [ ]:
train_raw_CI['var_names'][90:95].values

## Create new normalization file

In [ ]:
norm_RH_dataset = xr.open_dataset(path_norm_RH)
norm_BMSE_dataset = xr.open_dataset(path_norm_BMSE)
norm_LHF_nsDELQ_dataset = xr.open_dataset(path_norm_LHF_nsDELQ) 

In [ ]:
norm_dataset = xr.open_dataset(path_input_norm)

In [ ]:
new_norm_dataset = norm_dataset.copy()

### Coordinates

In [ ]:
var_names_full = norm_dataset['var_names'].values
var_names_full_single = norm_dataset['var_names_single'].values

In [ ]:
for i in range(30): var_names_full = np.append(var_names_full,'RH')
for i in range(30): var_names_full = np.append(var_names_full,'BMSE')
var_names_full = np.append(var_names_full,'LHF_nsDELQ')

var_names_full_single = np.append(var_names_full_single,'RH')
var_names_full_single = np.append(var_names_full_single,'BMSE')
var_names_full_single = np.append(var_names_full_single,'LHF_nsDELQ')

In [ ]:
var_names_full.shape

In [ ]:
var_names_full_single.shape

In [ ]:
new_coor = {}
new_coor['var_names'] = var_names_full
new_coor['var_names_single'] = var_names_full_single

### Data

#### Full profiles

In [ ]:
KEY = ['mean','std','min','max']

In [ ]:
norm_data = {}

In [ ]:
for key in KEY:
    norm_data[key] = norm_dataset[key].values

In [ ]:
norm_RH_dataset['var_names'][:30].values

In [ ]:
for key in KEY:
    norm_data[key] = np.append(norm_data[key],norm_RH_dataset[key][:30].values)

In [ ]:
norm_BMSE_dataset['var_names'][30:60].values

In [ ]:
for key in KEY:
    norm_data[key] = np.append(norm_data[key],norm_BMSE_dataset[key][30:60].values)

In [ ]:
norm_LHF_nsDELQ_dataset['var_names'][93].values

In [ ]:
for key in KEY:
    norm_data[key] = np.append(norm_data[key],norm_LHF_nsDELQ_dataset[key][93].values)

#### One std per variable

In [ ]:
key0 = 'std_by_var'

In [ ]:
norm_data[key0] = norm_dataset[key0].values

In [ ]:
norm_dataset[key0].values.shape

In [ ]:
norm_RH_dataset['var_names_single'][0].values

In [ ]:
norm_data[key0] = np.append(norm_data[key0],
                            norm_RH_dataset[key0][0].values)

In [ ]:
norm_BMSE_dataset['var_names_single'][1].values

In [ ]:
norm_data[key0] = np.append(norm_data[key0],
                            norm_BMSE_dataset[key0][1].values)

In [ ]:
norm_LHF_nsDELQ_dataset['var_names_single'][6].values

In [ ]:
norm_data[key0] = np.append(norm_data[key0],
                            norm_LHF_nsDELQ_dataset[key0][6].values)

In [ ]:
for key in norm_data.keys():
    print(key+str(norm_data[key].shape))

In [ ]:
norm_data_dict = {}

In [ ]:
for key in KEY:
    norm_data_dict[key] = (['var_names'],norm_data[key])
norm_data_dict[key0] = (['var_names_single'],norm_data[key0])

### Combine coordinates and data into a new xarray dataset

In [ ]:
new_norm = xr.Dataset(
    data_vars = norm_data_dict,
    coords = new_coor
)

### Check that new normalization file was created correctly

#### Full profiles

In [ ]:
new_norm['var_names'][-31:-1].values

In [ ]:
new_norm['min'][-31:-1].values

In [ ]:
norm_BMSE_dataset['var_names'][30:60].values

In [ ]:
norm_BMSE_dataset['min'][30:60].values

#### One std per variable

In [ ]:
new_norm['var_names_single'][-3].values

In [ ]:
new_norm['std_by_var'][-3].values

In [ ]:
norm_RH_dataset['var_names_single'][0].values

In [ ]:
norm_RH_dataset['std_by_var'][0].values

## Save new norm file

In [ ]:
new_norm

In [ ]:
norm_dataset

In [ ]:
new_norm.to_netcdf(path_save_dir+'2021_12_22_NORM_For_Nando_CI.nc',mode='w')

# Check that training is now stable

In [12]:
path_save_dir = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/Aqua_0K_ClimInv_withVBP/'

In [13]:
path_train = '2021_12_22_TRAIN_For_Nando_CI.nc'
path_newnorm = '2021_12_22_NORM_For_Nando_CI.nc'

In [14]:
test_train = xr.open_dataset(path_save_dir+path_train)

In [15]:
test_train['var_names'][90:95]

<xarray.DataArray 'var_names' (var_names: 5)>
array(['PS', 'SOLIN', 'SHFLX', 'LHF_nsDELQ', 'PHQ'], dtype=object)
Coordinates:
  * var_names  (var_names) object 'PS' 'SOLIN' 'SHFLX' 'LHF_nsDELQ' 'PHQ'

In [16]:
in_vars = ['RH','BMSE','PS', 'SOLIN', 'SHFLX', 'LHF_nsDELQ']
out_vars = ['PHQ','TPHYSTND','FSNT','FSNS','FLNT','FLNS','PRECT']

In [17]:
train_gen_Nando = DataGeneratorCI(
    data_fn = path_save_dir+path_train,
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = path_save_dir+path_newnorm,
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=N_batch
)

In [18]:
train_gen_Nando[0][0]

array([[-0.11563262, -0.09748988, -0.12243562, ...,  0.11276857,
        -0.00632469, -0.03836039],
       [-0.11563261, -0.09748984, -0.12243327, ...,  0.11292087,
        -0.00523675, -0.03212018],
       [-0.1156326 , -0.0974898 , -0.12243068, ...,  0.11315581,
        -0.00450536, -0.02770003],
       ...,
       [ 0.88424507,  0.86546891,  0.75225603, ..., -0.22988858,
        -0.00266731,  0.09164395],
       [ 0.88419446,  0.86494852,  0.75222398, ..., -0.22988858,
        -0.0051181 , -0.02362656],
       [ 0.88413834,  0.86444557,  0.75219506, ..., -0.22988858,
        -0.00200984,  0.11747147]])

In [19]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
densout = Dense(128, activation='linear')(inp)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (6):
    densout = Dense(128, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
dense_out = Dense(65, activation='linear')(densout)
model = tf.keras.models.Model(inp, dense_out)

In [20]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense (Dense)                (None, 128)               8320      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 128)               0     

In [21]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [22]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_12_22_Test_Nando'

In [23]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
Nep = 20
model.fit_generator(train_gen_Nando, epochs=Nep, validation_data=train_gen_Nando,\
                    callbacks=[earlyStopping, mcp_save_pos])

Epoch 1/20
5759/5759 [==============================] - 286s 50ms/step - loss: 899.5385 - val_loss: 739.3284
Epoch 2/20
5759/5759 [==============================] - 270s 47ms/step - loss: 723.8076 - val_loss: 701.5817
Epoch 3/20
5759/5759 [==============================] - 262s 45ms/step - loss: 696.2791 - val_loss: 685.8165
Epoch 4/20
5759/5759 [==============================] - 265s 46ms/step - loss: 681.0514 - val_loss: 676.0172
Epoch 5/20
5759/5759 [==============================] - 264s 46ms/step - loss: 670.7355 - val_loss: 665.3600
Epoch 6/20
5759/5759 [==============================] - 277s 48ms/step - loss: 662.9595 - val_loss: 659.0412
Epoch 7/20
5757/5759 [============================>.] - ETA: 0s - loss: 657.2372